<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# Speed Up DataFrame Operations w/ RAPIDS cuDF

## ETL Acceleration
A **DataFrame** is a 2-dimensional data structure used to represent data in a tabular format, like a spreadsheet or SQL table. Originally offered through the Python Data Analysis ([pandas](https://pandas.pydata.org/docs/)) library, DataFrames have become very popular for its familiar representation along with a robust set of features that are intuitive and expressive. 

Raw data often needs to be manipulated before it can be used for further purposes such as generating **Business Intelligence**, creating **Dashboard Visualization**, or training **Machine Learning** models. These preprocessing steps can include **filtering**, **merging**, **grouping**, and **aggregating**. 

Below is a typical data processing pipeline: 
<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/flow.png?raw=true' atl='flow' width=1080></p>

According to [studies](https://www.forbes.com/sites/gilpress/2016/03/23/data-preparation-most-time-consuming-least-enjoyable-data-science-task-survey-says/?sh=29f71b266f63), data preparation accounts for ~80% of the work for analysts. This could be due in part to the rapid increase in the size of data as well as the iterative nature of analytics. 

Recognizing this potential bottleneck, NVIDIA created [**cuDF**](https://docs.rapids.ai/api/cudf/stable/) that leverages GPU hardware and software to perform data manipulation tasks with parallel computing, **saving valuable time and resources**. The cuDF library is part of the larger [**RAPIDS**](https://rapids.ai/) data science framework that allows for the execution of **end-to-end analytics pipelines** entirely on GPUs. One of the focus for cuDF and its companion suite of open source software libraries is to provide syntax that is similar to their CPU counterparts, **making it easy to implement**. 

This notebook is intended to demonstrate speedup in data processing by moving common DataFrame operations to the GPU with minimal changes to existing code. 

### Environment Sanity Check
Check the output of `!nvidia-smi` to make sure you've been allocated a RAPIDS supported GPU such as Tesla T4, P4, or P100.

In [1]:
!nvidia-smi

Fri Mar 14 18:43:07 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   23C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Interactive Exercise

In [2]:
import numpy as np # for generating sample data

import pandas as df
# import cudf as df
import time # for clocking process times
import matplotlib.pyplot as plt # for visualizing results

class Timer: # creating a Timer helper class to measure execution time
  def __enter__(self):
    self.start=time.perf_counter()
    return self
  def __exit__(self, *args):
    self.end=time.perf_counter()
    self.interval=self.end-self.start

### Loading a Sample Data
We start our demonstration by generating two 2-dimensional arrays of random numbers - we've configured for sizeable arrays at 1MM rows by 50 columns each. Then they are converted to DataFrames using ```pandas.DataFrame()``` or ```cudf.DataFrame()```:

In [3]:
rows=1000000
columns=50

In [4]:
def load_data(): 
  data_a=np.random.randint(0, 100, (rows, columns))
  data_b=np.random.randint(0, 100, (rows, columns))
  dataframe_a=df.DataFrame(data_a, columns=[f'a_{i}' for i in range(columns)])
  dataframe_b=df.DataFrame(data_b, columns=[f'b_{i}' for i in range(columns)])
  return dataframe_a, dataframe_b

with Timer() as process_time: 
  dataframe_a, dataframe_b=load_data()

print(f'The loading process took {process_time.interval:.2f} seconds')
display(dataframe_a.tail(5))
display(dataframe_b.tail(5))

The loading process took 0.99 seconds


,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,...,a_40,a_41,a_42,a_43,a_44,a_45,a_46,a_47,a_48,a_49
999995,39,54,43,12,35,52,17,6,39,23,...,66,38,54,90,0,8,28,60,51,3
999996,4,95,49,17,53,29,89,77,4,13,...,73,65,63,18,9,68,14,31,72,89
999997,9,27,20,19,33,55,38,20,35,8,...,84,17,32,36,25,68,18,57,56,56
999998,81,91,74,55,68,63,41,27,73,52,...,49,4,16,64,96,22,87,31,0,15
999999,97,15,51,83,8,77,46,53,78,86,...,66,84,63,18,32,28,11,24,23,80


,b_0,b_1,b_2,b_3,b_4,b_5,b_6,b_7,b_8,b_9,...,b_40,b_41,b_42,b_43,b_44,b_45,b_46,b_47,b_48,b_49
999995,62,50,15,75,30,71,3,27,7,69,...,39,13,90,77,95,45,44,63,88,71
999996,33,27,96,95,10,10,99,25,42,68,...,48,90,19,77,7,98,4,42,54,88
999997,74,67,46,8,72,9,1,40,32,98,...,30,44,4,39,79,24,57,56,48,30
999998,83,64,55,31,65,46,33,41,11,34,...,63,70,46,91,81,81,82,51,19,6
999999,51,50,36,0,20,2,17,42,48,64,...,99,27,6,56,69,31,18,77,53,35


<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/check.png?raw=true' width=720 atl='check'></p>

We created two DataFrames, _dataframe_a_ and _dataframe_b_ that are 1000000 rows by 50 columns (col_1, col_2, ... col_48, col_49) each. 

### Merging Data
Sometimes data can come from multiple sources and need to be merged into one with ```DataFrame.merge()```. For example, a typical retail data storage infrastructure may include a customer table and separate transaction and product tables. Merging the data allows the correct details to be included in a single DataFrame to get the insight needed. 

In [5]:
def merge_data(left_df, right_df):
  combined_df=df.merge(left_df, right_df, left_index=True, right_index=True)
  return combined_df

with Timer() as process_time: 
  combined_df=merge_data(dataframe_a, dataframe_b)

print(f'The merging process took {process_time.interval:.2f} seconds')
display(combined_df.head())

The merging process took 1.31 seconds


,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,...,b_40,b_41,b_42,b_43,b_44,b_45,b_46,b_47,b_48,b_49
0,87,17,30,2,92,98,15,19,43,46,...,99,25,82,15,55,60,9,17,16,70
1,72,65,80,26,41,88,98,79,4,65,...,1,56,70,94,27,90,29,43,22,12
2,35,55,0,74,39,87,93,38,95,40,...,70,26,47,92,34,49,12,78,94,14
3,19,64,25,93,1,63,82,6,7,89,...,68,60,81,18,58,62,93,50,78,98
4,1,26,99,56,11,49,77,45,8,0,...,19,92,73,90,7,25,15,47,82,93


<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/check.png?raw=true' width=720 atl='check'></p>

We merged two DataFrames, _dataframe_a_ and _dataframe_b_ on their _index_ into one larger DataFrame that is 1000000 rows by 100 columns (a_0, a_1, ..., b_48, b_49). 

### Summarize
Exploring data begins with **descriptive statistics**, which often involves finding the **central tendency** and **dispersion**. They are a quick way to summarize distributions. Measures of central tendency includes the mean, median, and mode - they are used to describe the center of a set of data values. Measures of dispersion include variance and standard deviation - they are used to describe the degree to which data is distributed around the center. We can quickly perform simple descriptive statistics with the ```DataFrame.describe()``` method. 

In [6]:
def summarize(dataframe):
  summary_df=dataframe.describe()
  return summary_df

with Timer() as process_time: 
  summary_df=summarize(combined_df)

print(f'The summarizing process took {process_time.interval:.2f} seconds')
display(summary_df)

The summarizing process took 4.76 seconds


,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,...,b_40,b_41,b_42,b_43,b_44,b_45,b_46,b_47,b_48,b_49
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,...,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,49.486222,49.526124,49.527647,49.485708,49.491233,49.515229,49.554284,49.490764,49.516843,49.500240,...,49.492779,49.498878,49.554114,49.489541,49.469201,49.510828,49.504097,49.524133,49.542504,49.514471
std,28.849019,28.881237,28.873595,28.866101,28.850891,28.865636,28.866429,28.869602,28.862507,28.869047,...,28.867084,28.871197,28.869659,28.867028,28.861531,28.865835,28.872735,28.873473,28.870686,28.871396
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,24.000000,25.000000,25.000000,24.000000,24.000000,24.000000,25.000000,24.000000,24.000000,24.000000,...,24.000000,24.000000,25.000000,24.000000,24.000000,24.000000,24.000000,25.000000,25.000000,25.000000
50%,49.000000,50.000000,50.000000,49.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,49.000000,50.000000,50.000000,49.000000,50.000000,50.000000,50.000000,50.000000,50.000000
75%,74.000000,75.000000,75.000000,74.000000,74.000000,74.000000,75.000000,74.000000,75.000000,74.000000,...,74.000000,75.000000,75.000000,74.000000,74.000000,75.000000,74.000000,75.000000,75.000000,75.000000
max,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,...,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000


<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/check.png?raw=true' width=720 atl='check'></p>

Since this is a sample data set, we see that each of columns/features (a_0, a_1, ..., b_48, b_49) have 1000000 values with an average ~50 and standard deviation of ~30

### Correlation - Exploring Relationships
We might be interested in finding relationships/dependencies between two or more variables through their correlation with ```DataFrame.corr()```. Correlation is a number between -1 and 1 that describes the strength of the association between two variables. Two variables with a correlation of 1 suggests that they change together in the same direction while a correlation of -1 suggests that they change together in the opposite direction. 

In [7]:
def correlation(dataframe): 
  corr_df=dataframe.corr()
  return corr_df

with Timer() as process_time: 
  corr_df=correlation(combined_df)

print(f'The correlation process took {process_time.interval:.2f} seconds')
display(corr_df.head())

The correlation process took 23.91 seconds


,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,...,b_40,b_41,b_42,b_43,b_44,b_45,b_46,b_47,b_48,b_49
a_0,1.000000,-0.000290,0.000538,-0.001089,-0.001322,-0.000240,-0.000088,-0.000361,-0.000554,0.001496,...,-0.000310,0.000428,0.000113,-0.000853,0.002017,-0.000937,0.000732,-0.002088,-0.000235,0.000216
a_1,-0.000290,1.000000,-0.000733,-0.001419,-0.001081,-0.000244,-0.001207,0.000971,-0.000313,-0.000566,...,-0.000211,0.000894,-0.001005,-0.000605,0.001153,-0.001486,-0.001241,-0.000427,-0.001214,0.000806
a_2,0.000538,-0.000733,1.000000,0.000451,-0.000039,-0.000136,-0.000925,-0.000257,0.003110,-0.000667,...,-0.000358,0.000661,0.000588,-0.000420,-0.000072,-0.000775,0.000420,-0.000472,0.000415,-0.000542
a_3,-0.001089,-0.001419,0.000451,1.000000,0.001101,-0.000024,0.000639,-0.000399,0.000734,0.001785,...,0.001339,0.001011,0.000946,-0.000048,0.000102,-0.000901,-0.000468,-0.001063,0.000471,0.001228
a_4,-0.001322,-0.001081,-0.000039,0.001101,1.000000,-0.000952,0.000525,0.000136,-0.000126,-0.001064,...,-0.000365,0.000084,0.000191,0.002430,-0.000063,-0.000047,0.000779,-0.000093,-0.000098,-0.000850


<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/check.png?raw=true' width=720 atl='check'></p>

The resulting cross tabulation shows that each column/feature (a_0, a_1, ..., b_48, b_49) have a perfect correlation (1) with itself and is not correlated (~0) with each other. 

### Grouping
We can compare subsets of the data to explore the significance of categories and classes with the ```DataFrame.groupby()``` method. We can even group continuous data values into a smaller number of bins with ```pandas.cut()``` or ```cudf.cut()``` to simplify our analysis. The groupings usually follow an aggregation such as mean or count. For example, we can group our data into 5 equidistant bins based on their sequential index. 

In [8]:
def groupby_summarize(dataframe):
    dataframe['group']=dataframe.index
    dataframe['group']=df.cut(dataframe['group'], 5)
    group_describe_df=dataframe.groupby('group').mean().reset_index(drop=True)
    return group_describe_df

with Timer() as process_time: 
    group_describe_df=groupby_summarize(combined_df)

print(f'The grouping process took {process_time.interval:.2f} seconds')
display(group_describe_df)

The grouping process took 1.09 seconds


,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,...,b_40,b_41,b_42,b_43,b_44,b_45,b_46,b_47,b_48,b_49
0,49.412520,49.549715,49.496020,49.598845,49.495965,49.519800,49.569340,49.453490,49.455210,49.511565,...,49.432215,49.589875,49.622340,49.426930,49.491955,49.437390,49.497595,49.485110,49.578940,49.463270
1,49.453455,49.576010,49.559625,49.573180,49.506475,49.448235,49.607785,49.549805,49.613630,49.543790,...,49.507590,49.503995,49.493320,49.576175,49.513970,49.509410,49.547115,49.447650,49.569765,49.577440
2,49.411955,49.499005,49.511560,49.368710,49.460610,49.588395,49.535035,49.451110,49.599125,49.471510,...,49.543230,49.452190,49.568295,49.439370,49.528870,49.480765,49.459800,49.481835,49.495995,49.539840
3,49.610635,49.483215,49.466965,49.476475,49.437255,49.460245,49.531745,49.515695,49.450980,49.449775,...,49.537705,49.522865,49.443870,49.602505,49.428290,49.574365,49.458020,49.622760,49.532435,49.499300
4,49.542545,49.522675,49.604065,49.411330,49.555860,49.559470,49.527515,49.483720,49.465270,49.524560,...,49.443155,49.425465,49.642745,49.402725,49.382920,49.552210,49.557955,49.583310,49.535385,49.492505


<p><img src='https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/check.png?raw=true' width=720 atl='check'></p>

The resulting DataFrame shows that each group maintains an average of ~50 for each column/feature (a_0, a_1, ..., b_48, b_49) as expected for this sample data. 

### Putting it together
We can measure the total elapsed time for this sample data processing workflow. 

In [9]:
def pipeline():
    performance={}
    with Timer() as process_time: 
        dataframe_a, dataframe_b=load_data()
    performance['load data']=process_time.interval
    with Timer() as process_time: 
        combined_df=merge_data(dataframe_a, dataframe_b)
    performance['merge data']=process_time.interval
    with Timer() as process_time: 
        summarize(combined_df)
    performance['summarize']=process_time.interval
    with Timer() as process_time: 
        correlation(combined_df)
    performance['correlation']=process_time.interval
    with Timer() as process_time: 
        groupby_summarize(combined_df)
    performance['groupby & summarize']=process_time.interval
    if df.__name__=='cudf': 
        df.DataFrame([performance], index=['gpu']).to_pandas().plot(kind='bar', stacked=True)
    else: 
        df.DataFrame([performance], index=['cpu']).plot(kind='bar', stacked=True)
    return None

### Timing the Pipeline on CPU

In [ ]:
import pandas as df
pipeline()

### Switching to GPU
Traditionally, these tasks are frequently done (as we did) using the popular [**pandas**](https://pandas.pydata.org/) library, which only runs on a single CPU. NVIDIA's [**cuDF**](https://docs.rapids.ai/api/cudf/stable/) library was built with the users in mind - by offering nearly identical syntax to its CPU counterpart, developers only have to make few changes to their existing code to take advantage of its capabilities. 

In [ ]:
import cudf as df

**That's it!** cuDF uses nearly identical syntax to the familiar pandas API. **Brilliant!** It's worth noting that there are some features that are unique to each library, but conviniently there are a lot of overlaps. 

In [ ]:
pipeline()

### Comparing Results
In a trial run, **cuDF** completed the data processing tasks in nearly 10x faster than **pandas**. The expectations is that the speedup will be even more significant as the size of the data becomes largers. Feel free to give it a try by modifying the dimensions of the data above. 

![result](https://github.com/NVDLI/notebooks/blob/kl/cudf_speed_up/images/result.png?raw=true)

## Conclusion
Congratulations on completing the notebook! Want to learn more about cuDF and the rest of the RAPIDS framework? Check out the follow-up to this course, [Accelerating End-to-End Data Science Workflows]('https://courses.nvidia.com/courses/course-v1:DLI+S-DS-01+V1/about') or our other online courses at [NVIDIA DLI]('https://www.nvidia.com/en-us/training/online/').